# Pull Content using a from a workdoc Folder
---

In [24]:
import boto3
import requests
import boto3
import re

def to_file_name_friendly_string(string):
    # Remove any characters that are not alphanumeric, underscore or hyphen
    file_name_friendly_string = re.sub(r'[^\w-]+', '', string)
    # Replace spaces with underscores
    file_name_friendly_string = file_name_friendly_string.replace(' ', '_')
    # Convert to lowercase
    file_name_friendly_string = file_name_friendly_string.lower()
    return file_name_friendly_string

BUCKET_NAME = 's3-to-gumps-workdoc'
TEST_OBJ = 'Sonde.pdf'
PROD_AP_FOLDER_ID = '81524e3f1939308697860fa97638399d31afbd3bcc78ecbfa81e23767565c49d'
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)
s3_resource = session.resource('s3')
s3_client = session.client('s3')
s3_resource = session.resource('s3')
workdocs_client = session.client('workdocs')
s3_obj = s3_resource.Object(BUCKET_NAME, TEST_OBJ)
s3_obj_response_body=s3_obj.get()['Body'].read()
s3_obj_response=s3_obj.get()
obj_type= s3_obj_response['ContentType']

In [25]:

session_workdoc_acc = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)

workdocs_client = session_workdoc_acc.client('workdocs')

In [26]:
workdocs_client = session.client('workdocs')
response = workdocs_client.describe_folder_contents(FolderId=PROD_AP_FOLDER_ID)
existing_folders = [(folder['Name'], folder['Id']) for folder in response['Folders']]

while response['Marker']:
    response = workdocs_client.describe_folder_contents(FolderId=PROD_AP_FOLDER_ID, Marker=response['Marker'])
    existing_folders.extend([(folder['Name'], folder['Id']) for folder in response['Folders']])

KeyError: 'Marker'

In [ ]:
print('# of Folders:',len(existing_folders))


# Pull Content using a from a workdoc Folder then upload to a csv format
---


In [ ]:
import csv

# Define the list of sets
NAME_OF_FOLDERS_IN_AP_CSV = 'ap_folder_name_n_id.csv'

# Open the output file
with open(NAME_OF_FOLDERS_IN_AP_CSV, 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)
    
    # Write the header row (optional)
    writer.writerow(['Folder Name', 'Folder ID'])
    
    # Write each set as a row in the CSV file
    for row in existing_folders:
        writer.writerow(list(row))

In [5]:
# Pull current Vendors tables

import pandas as pd
import numpy as np

def convert_to_dynamodb_friendly(string):
    """Converts any string to a DynamoDB-friendly string"""
    return string.replace('\\', '\\\\').replace('\n', '\\n').replace('\r', '\\r').replace('\t', '\\t').replace('\'', '\\\'').replace('\"', '\\"').replace('"', '')

df_vendor_dev = pd.read_csv('dev_vendor_tables.csv')
df_vendor_prod = pd.read_csv('ap_folder_name_n_id.csv')

In [ ]:
df_vendor_dev.head(5)

In [ ]:
df_vendor_prod.head(5)

In [ ]:
new_df = df_vendor_dev.merge(df_vendor_prod, how='right', left_on='workdocs_folder_name', right_on='Folder Name')

In [ ]:
new_df[['alt_names','exclude','known_email_domain','manual_process','workdocs_folder_name','Folder ID']].head(3)

In [ ]:
new_df[new_df['alt_names'].isnull()]
new_df.loc[new_df['alt_names'].isnull(), 'exclude'] = False
new_df.loc[new_df['alt_names'].isnull(), 'manual_process'] = False



In [ ]:
new_df.loc[new_df['alt_names'].isnull(), 'alt_names'] = new_df['Folder Name'].apply(convert_to_dynamodb_friendly)


In [ ]:
new_df.head(3)

## Create new folders within a workdocs folder
---

In [ ]:
## INPUT
TARGETED_FOLDER_ID = '3095d19b60533b9e374205aa6950f477b238490ca920e4ffa114428d5c5b284a' # Targeted Folder Id
LIST_OF_FOLDER_NAMES = new_df['Folder Name'].tolist() # List of folder names

LIST_OF_FOLDER_NAMES_WITH_NEW_ID = []
for folder in LIST_OF_FOLDER_NAMES:
    try:
        response = workdocs_client.create_folder(
            Name=folder,
            ParentFolderId=TARGETED_FOLDER_ID
        )
        LIST_OF_FOLDER_NAMES_WITH_NEW_ID.append((folder, response['Metadata']['Id']))
    except:
        print('Error: on folder', folder)

In [ ]:

new_folder_df = pd.DataFrame(LIST_OF_FOLDER_NAMES_WITH_NEW_ID, columns=['Folder Name', 'New Folder ID'])


In [ ]:
df_pre = new_df.merge(new_folder_df, how='left', on='Folder Name')

In [ ]:
df_pre.to_csv('vendor_tables.csv', index=False)

## Upload Vendors to Database using GraphQL
---

#### Requirements
- GraphQL endpoint and api key
- Vendor csv file

In [6]:
import pandas as pd
import numpy as np
vendors_df = pd.read_csv('vendor_Tables.csv')

In [7]:
vendors_df.head(2)

,id,__typename,_lastChangedAt,_version,alt_names,createdAt,exclude,known_email_domain,manual_process,updatedAt,workdocs_folder_id,workdocs_folder_name,Folder Name,Folder ID,New Folder ID
0,67479432-41d2-4fda-9c3d-5a12b287cae1,Vendor,1.677312e+12,1.0,VILLA & HOUSE,2023-02-25T08:02:44.851Z,False,NaN,False,2023-02-25T08:02:44.851Z,776c38dca2b9c24ddbff829ffa04b37811b0155b5f5359...,Bungalow,Bungalow,538107bd1077a05ce44fba4a67d1fc4c4d45703a3e68ae...,dcb350b40f23d8cb96a4be1df86bc4170fae661383f826...
1,a7a5d1ed-eaae-4da2-a1eb-7f93aa25e68c,Vendor,1.677312e+12,1.0,Halcyon Days Limited,2023-02-25T08:02:50.938Z,False,angela.hodgkinson@halcyondays.co.uk,False,2023-02-25T08:02:50.938Z,128c0d44cff625fe249c3ceb7079770f58326b81fa49a5...,Halycon Days,Halycon Days,a6a7064e128ebf312f0da2c9e680866511c8a0e9d5d1b2...,758290572038395ff7b5e9aa373ecc51807bb525cd964b...


In [8]:
vendors_df['workdocs_folder_name'] = vendors_df['workdocs_folder_name'].apply(convert_to_dynamodb_friendly)
vendors_df['alt_names'] = vendors_df['alt_names'].apply(convert_to_dynamodb_friendly)

In [9]:
vendors_df = vendors_df[['alt_names','exclude','known_email_domain','manual_process','workdocs_folder_name','Folder ID']]

In [10]:
vendors_df.to_csv('csv_list_prod.csv')

In [11]:
import requests
import boto3
accessToken = "da2-slcrizhc3rcbvdatpwmxq5ueqe"
endpoint = f"https://vkq5mcz3xfec7fcpzlw74fxq4m.appsync-api.us-east-1.amazonaws.com/graphql"

def query_graphql_ap_inbox_db(accessToken, endpoint, query):
    # establish a session with requests session
    session = requests.Session()
    # As found in AWS Appsync under Settings for your endpoint.
    APPSYNC_API_ENDPOINT_URL = endpoint
    # Now we can simply post the request...
    response = session.request(
        url=APPSYNC_API_ENDPOINT_URL,
        method='POST',
        headers={'x-api-key': accessToken},
        json={'query': query}
    )
    return response.json()


In [18]:
import csv

with open('csv_list_prod.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    csv_list = [tuple(row) for row in reader]

In [19]:
csv_list[0]

('',
 'alt_names',
 'exclude',
 'known_email_domain',
 'manual_process',
 'workdocs_folder_name',
 'Folder ID')

In [20]:
error_list = []
for _, known_name, exclude, known_email_domain, manual_rev, workdocs_folder_name, workdocs_folder_id  in csv_list[1:]:
    if manual_rev == 'True':
        manual_rev = 'true'
    else:
        manual_rev = 'false'
    if exclude == 'yes':
        exclude = 'true'
    else:
        exclude = 'false'

    graphql_query = f"""
    mutation MyMutation($alt_names: String = "{known_name}", 
                        $workdocs_folder_id: String = "{workdocs_folder_id}",
                        $known_email_domain: String = "{known_email_domain}",
                        $workdocs_folder_name: String = "{workdocs_folder_name}") {{
    createVendor(input: {{alt_names: $alt_names,
                         manual_process: {manual_rev},
                         workdocs_folder_id: $workdocs_folder_id,
                         known_email_domain: $known_email_domain,
                         exclude: {exclude},
                         workdocs_folder_name: $workdocs_folder_name,
                        workdocs_folder_id: $workdocs_folder_id}}) {{
        id
        }}
        }}



    """
    response = query_graphql_ap_inbox_db(accessToken, endpoint, graphql_query)
    try:
        if response['errors']:
            error_list.append((_, known_name, exclude, known_email_domain, manual_rev, workdocs_folder_name, workdocs_folder_id))
    except:
        pass
    print(response)

{'data': {'createVendor': {'id': 'fc68abed-b350-4ee6-ab30-2b2804709ab1'}}}
{'data': {'createVendor': {'id': 'fe4adf52-5c6a-4055-bf81-bcb5fa58e1bb'}}}
{'data': {'createVendor': {'id': '5146e839-d39c-452d-93fb-1f3c6b709f3a'}}}
{'data': {'createVendor': {'id': 'c61bf36f-c1d0-498c-bf01-cf13caddae58'}}}
{'data': {'createVendor': {'id': '85847556-418f-4811-a2fa-223ad6499d73'}}}
{'data': {'createVendor': {'id': '38d1c7f2-3b8f-4224-9400-a646cc742488'}}}
{'data': {'createVendor': {'id': '4b1e2fb4-6f99-44d6-828f-20b37ca59e77'}}}
{'data': {'createVendor': {'id': '4873044b-5a29-462c-bff9-69d97d173f95'}}}
{'data': {'createVendor': {'id': 'cda845fa-a11a-4949-88b0-388d8547d366'}}}
{'data': {'createVendor': {'id': 'fd3f837d-a5b5-4b4b-ab47-0a5720dcb932'}}}
{'data': {'createVendor': {'id': '35ab5048-4a8a-4408-89c2-381608c988d5'}}}
{'data': {'createVendor': {'id': '761527c6-cda5-4f26-9595-645ea703c590'}}}
{'data': {'createVendor': {'id': 'f4e3d350-4f65-4f90-a96e-17c3c501546f'}}}
{'data': {'createVendor':

In [21]:
error_list

[]

In [35]:
import json
import os
import boto3
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

STATE_MACHINE_ARN = 'arn:aws:states:us-east-1:570685116845:stateMachine:ap-inbox-automation-preprod'
event = {
   "Records":[
      {
         "EventSource":"aws:sns",
         "EventVersion":"1.0",
         "EventSubscriptionArn":"arn:aws:sns:us-east-1:570685116845:ap-inbox-preprod-SavedInS3Notification-JOXaEJvRhtld:47965192-b5d2-4570-be97-fad48e924799",
         "Sns":{
            "Type":"Notification",
            "MessageId":"93fcf6f6-505a-5093-b070-aa36d091043c",
            "TopicArn":"arn:aws:sns:us-east-1:570685116845:ap-inbox-preprod-SavedInS3Notification-JOXaEJvRhtld",
            "Subject":"Amazon SES Email Receipt Subscription Notification",
            "Message":"{\"notificationType\":\"Received\",\"mail\":{\"timestamp\":\"2023-04-24T13:05:15.116Z\",\"source\":\"no-reply-aws@amazon.com\",\"messageId\":\"AMAZON_SES_SETUP_NOTIFICATION\",\"destination\":[\"recipient@example.com\"]},\"receipt\":{\"timestamp\":\"2023-04-24T13:05:15.116Z\",\"processingTimeMillis\":0,\"recipients\":[\"recipient@example.com\"],\"spamVerdict\":{\"status\":\"PASS\"},\"virusVerdict\":{\"status\":\"PASS\"},\"spfVerdict\":{\"status\":\"PASS\"},\"dkimVerdict\":{\"status\":\"PASS\"},\"action\":{\"type\":\"S3\",\"topicArn\":\"arn:aws:sns:us-east-1:570685116845:ap-inbox-preprod-SavedInS3Notification-JOXaEJvRhtld\",\"bucketName\":\"ap-inbox-preprod-receivedemailstorage-gphsonnba9lq\",\"objectKey\":\"AMAZON_SES_SETUP_NOTIFICATION\"}}}",
            "Timestamp":"2023-04-24T13:05:15.139Z",
            "SignatureVersion":"1",
            "Signature":"yv8LvH2VhW4ni+64fPze3E4TZMx3GUbE0ZUud2xNMYOTBged6IlTHjfWM/013lVEr9STJe73MvrROQgJ4VWc8qN2wMtv1sHpegrtQ9ZyXic8daSzk0GF2jcU4+0C8+We9tE4reXYU1BWzArxTIKY85VS8GzBcUtBRrfP6WP1eTI1io3GVao/ojCn5t96WrurMnNRMgreDiVzZAR7NuDIt+yEHsXs31E1lsPpE1wv6NN4yQPlY5zzYtcPLonHRd7PJOJInDOXwpnDfJqajH+t37Vm10h3t3OAzyJrAQ5o5Vwkvxzssw+FlwVQaH6eAYnnVH8/jgUwA60ZCszYJcsELg==",
            "SigningCertUrl":"https://sns.us-east-1.amazonaws.com/SimpleNotificationService-56e67fcb41f6fec09b0196692625d385.pem",
            "UnsubscribeUrl":"https://sns.us-east-1.amazonaws.com/?Action=Unsubscribe&SubscriptionArn=arn:aws:sns:us-east-1:570685116845:ap-inbox-preprod-SavedInS3Notification-JOXaEJvRhtld:47965192-b5d2-4570-be97-fad48e924799",
            "MessageAttributes":{
               
            }
         }
      }
   ]
}
stepfunctions = boto3.client('stepfunctions')
response = stepfunctions.start_execution(
    stateMachineArn= STATE_MACHINE_ARN,
    input= json.dumps(event, cls=DateTimeEncoder),
    )   